## ДЗ_3

### Задача: поиск похожих по эмбеддингам.

Скачиваем датасет:
Для выполнения 3 задания можно взять данные отсюда https://github.com/IlyaGusev/gazeta

### Необходимо сделать:

1. На основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к (примеру 5-ть), ваш метод должен возвращать 5-ть ближайших статей к этому запросу).

2. Проверить насколько хорошо работают подходы

#### Сначала импортируем необходимые библиотеки и загрузим данные.

In [2]:
!pip install natasha stop_words annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 9.6 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 kB 5.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 269.6 kB/s eta 0:00:0000:01
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 598.8 kB/s eta 0:00:00a 0:00:01
  Using cached razdel-0.5.0-py3-none-any.whl (21 kB)
  Preparing metadata (setup.py) ... done
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Using cached docopt-0.6.2-py2.py3-none-any.whl
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32893 sha256=e1c256a84b036c90558460d0029e34c70b28caf1e90181380f714dd0f71081e6
  Stored in directory: /Users/mac/Library/Caches/pip/wheels

In [4]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 641.7 kB/s eta 0:00:00a 0:00:01


In [26]:
import numpy as np
import pandas as pd

import string
import pickle
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

In [6]:
!wget https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz

--2022-10-18 16:13:17--  https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Распознаётся www.dropbox.com (www.dropbox.com)… 2620:100:6026:18::a27d:4612, 162.125.70.18
Подключение к www.dropbox.com (www.dropbox.com)|2620:100:6026:18::a27d:4612|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: /s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz [переход]
--2022-10-18 16:13:17--  https://www.dropbox.com/s/raw/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz
Повторное использование соединения с [www.dropbox.com]:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://ucfb671d134ad9a68b6ad13c065d.dl.dropboxusercontent.com/cd/0/inline/BvDAJZQM27CmTpjIGTc48Djt-y6yo5Fhu1zieuf-k-s8EVlswPDgl3MDLOT-tH4aNCmjLBcQN3whjimeLBTRFMQXZ5smBJ1w3KjKDbu4EeiwsdzUnhhxKwhM9XhzXAaW2fAyJ2WJnsrYrOjBkYod8Tu1ujoTVk7-x3Z_4a7hvWzv8A/file# [переход]
--2022-10-18 16:13:18--  https://ucfb671d134ad9a68b6ad13c065d.dl.dropboxusercontent.com/cd/0/inline/BvDAJZQM27CmTpjIGTc48Djt-y6yo5

In [7]:
!tar -xzvf gazeta_jsonl.tar.gz

x gazeta_train.jsonl
x gazeta_test.jsonl
x gazeta_val.jsonl


**Объединим данные в один датафрейм**

In [8]:
df_train = pd.read_json(path_or_buf='gazeta_train.jsonl', lines=True)
df_val = pd.read_json(path_or_buf='gazeta_val.jsonl', lines=True)
df_test = pd.read_json(path_or_buf='gazeta_test.jsonl', lines=True)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
df.head()

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63435 entries, 0 to 63434
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   url                   63435 non-null  object        
 1   text                  63435 non-null  object        
 2   title                 63435 non-null  object        
 3   summary               63435 non-null  object        
 4   date                  63435 non-null  datetime64[ns]
 5   preprocessed_summary  63435 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 2.9+ MB


In [43]:
df.shape

(63435, 6)

**Посмотрим за какой временной период собраны статьи в нашем корпусе.**

In [67]:
df.describe(datetime_is_numeric=True)

,date
count,63435
mean,2015-08-28 15:32:39.718688256
min,2010-06-01 10:35:49
25%,2012-04-10 09:49:32.500000
50%,2016-01-22 19:21:00
75%,2019-02-06 09:02:14
max,2020-03-23 22:16:23


Посмотрим на размеры текстов статей и на размеры резюме (summary) . к ним 

In [9]:
text_lengths = df['text'].apply(len)
summary_lengths = df['summary'].apply(len)

print(f'Средняя длина текста статьи - {np.average(text_lengths):.0f} слов.')
print(f'Средняя длина саммари - {np.average(summary_lengths):.0f} слов.')

Средняя длина текста статьи - 4519 слов.
Средняя длина саммари - 312 слов.


Напишем функцию для предварительной подготовки текста.

In [10]:
morpher = MorphAnalyzer()
stopwords = set(get_stop_words("ru"))
punkt = set(string.punctuation)

def preprocess_txt(line, morpher=morpher, sw=stopwords, punkt=punkt):
    spls = "".join(i for i in line.strip() if i not in punkt).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [11]:
df['preprocessed_summary'] = df['summary'].apply(preprocess_txt)
df.head()

,url,text,title,summary,date,preprocessed_summary
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39,"[2011, россия, уйти, 80, млрд, считать, минэко..."
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09,"[юлия, соловьёв, эксдиректор, холдинг, «профме..."
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14,"[суд, арестовать, месяц, четверо, эксчиновник,..."
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50,"[мнение, предприниматель, повод, вступление, в..."
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01,"[канал, завершиться, сезон, шоу, «голос», побе..."


In [15]:
# df['preprocessed_text'] = df['text'].apply(preprocess_txt)
# df.head()

KeyboardInterrupt: 

**Размер корпуса "text" оказался очень велик. Компьютер не справился с ним. Можно попробовать ограничить максимальное количество слов в статьях.**

**Мы попробуем воспользоваться другим методом. Для обучения моделей будет использовать содержимое раздела "summary". Так как он соответствует теме статьи и полностью отражает общую суть материалов, но при этом значительно короче, обучение будет происходить намного быстрее.**

***В том числе проверим нашу гипотезу.***

Создадим модели, используя предобработанные данные. Настройки оставим по умолчанию.

In [28]:
window_size = 5
vector_size = 300

modelW2V = Word2Vec(sentences=df['preprocessed_summary'], 
                    vector_size=vector_size, window=window_size, 
                    min_count=1)
modelFT = FastText(sentences=df['preprocessed_summary'], 
                   vector_size=vector_size, window=window_size, 
                   min_count=1)

Обучим модели и создадим функцию для получения ближайших статей с помощью построенных моделей.

In [45]:
w2v_index = annoy.AnnoyIndex(vector_size, 'angular')
ft_index = annoy.AnnoyIndex(vector_size, 'angular')

index_map = {}
counter = 0


for i in range(len(df['preprocessed_summary'])):
    n_w2v = 0
    n_ft = 0
    index_map[i] = df['text'][i]
    
    vector_w2v = np.zeros(vector_size)
    vector_ft = np.zeros(vector_size)
    for word in df['preprocessed_summary'][i]:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(i, vector_w2v)
    ft_index.add_item(i, vector_ft)


w2v_index.build(10)
ft_index.build(10)

True

In [30]:
def get_response(question, index, model, index_map, nn=5):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, nn, )
    return [index_map[i] for i in answers]

In [36]:
TEXT = "Новости экономики"

In [37]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['В новейшей российской практике для финансового рынка к разряду «вербальных интервенций» вполне можно относить «профессиональные суждения регулятора», то есть высказывания руководителей Центробоанка. 12 июля, выступая на радиостанции «Бизнес ФМ», глава Центробанка фактически определила судьбу «Югры»: «У банка уже, во-первых, возникли проблемы, поэтому мы ввели туда временную администрацию — она сейчас работает и должна будет оценить масштаб проблем, получить доступ к первоисточникам. После этого будет принято соответствующее решение. Конечно, нам нужно расширять сферу применения профессионального суждения, что, я знаю, многим моим коллегам за рубежом позволяет гораздо быстрее принимать решения, когда у банка возникают проблемы и у регулятора есть право обоснованного, профессионального, но суждения». «Суждение регулятора» было понято настолько верно, что все действия руководства «Югры» — от обращения к ЦБ с планом по самостоятельному выводу банка из кризиса и выплаты денег вкладчикам в

In [38]:
get_response(TEXT, ft_index, modelFT, index_map)

['Рынок недвижимости в Дубае сейчас характеризуется активным ростом цен и интересом со стороны иностранных инвесторов. После падения цен на жилье в Дубае в 2009 году рынок недвижимости переживает расцвет, в то время как в Абу-Даби, столице Объединенных Арабских Эмиратов, аналогичного восстановления рынка не наблюдается, сообщает аналитический портал Global PropertyGuide.com. За год (ноябрь 2011 — ноябрь 2012 года), по данным портала, цены в Дубае выросли на 18,26% с поправкой на инфляцию. Есть и другие данные: цены на квартиры в Дубае за последний год выросли на 42% (сентябрь 2012 — сентябрь 2013 года), следует из отчета местной компании Asteco. По оценкам экспертов, снижение цен в 2009–2010 годах способствовало росту спроса и интереса со стороны инвесторов, в том числе из стран Ближнего Востока, Северной Африки, Индийского полуострова и Центральной Азии. Из-за нестабильной ситуации в этом секторе сформировался отложенный спрос, что также объясняет текущий интерес инвесторов к этому ры

In [57]:
TEXT = "Сырьевые рынки"

In [58]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Согласно данным опубликованного отчета компании, чистая прибыль Ford в 2012 году составила $1,6 млрд. Тем не менее аналитики компании предсказывают, что потери Ford в Европе в 2013 году составят около $2 млрд, превысив таким образом данный ранее прогноз ($1,75 млрд). На фоне публикации невысоких результатов финансовой деятельности Ford в четвертом квартале 2012 года акции компании упали на 5%. В целом годовая выручка Ford сократилась на 1,5%, до $134,3 млрд. Операционная прибыль уменьшилась на 10% (до $7,9 млрд), а чистая прибыль — на 72% (до $5,6 млрд). При этом сокращение чистой прибыли составило всего 5%, если не учитывать феноменальный рост, который компания показала в четвертом квартале 2011 года: именно этот показатель и создает такую разницу. При этом компания установила рекорд доналоговой квартальной прибыли за последние 10 лет — этот показатель составил $1,7 млрд. Этому поспособствовал высокий уровень продаж пикапов и внедорожников Ford на территории США, а также повышение ц

In [59]:
get_response(TEXT, ft_index, modelFT, index_map)

['Южнокорейский производитель смартфонов Samsung предупредил о риске замедления темпов продаж, что может оказать негативное влияние на производство. На развитых рынках смартфонов уже много, а на развивающихся будут появляться новые по более низкой цене. Корпорация также опасается замедленного восстановления экономики в связи с кризисом и не увеличивает объем капитальных затрат на текущий год. Несмотря на рост продаж смартфонов в четвертом квартале 2012 года, который привел к тому, что чистая прибыль Samsung выросла за этот период на 76%, до 7,04 трлн вон ($6,6 млрд), корпорация предупредила, что в этом году на бизнес может оказать негативное влияние ценовая конкуренция, а также замедление спроса на развитых рынках. По итогам четвертого квартала консолидированная выручка Samsung выросла на 7%, до 56,06 трлн вон ($52 млрд), а операционная прибыль — на 10%, до 8,84 трлн вон ($8,2 млрд). Выручка Samsung от мобильного подразделения выросла по итогам четвертого квартала на 4%, до 27,23 трлн 

In [60]:
TEXT = "Нефтяные рынки"

In [61]:
get_response(TEXT, w2v_index, modelW2V, index_map)

["Соотношение цены акции к прогнозируемой прибыли (Р/Е) для компаний финансового сектора из индекса Standard & Poor's 500 в сентябре составляет 13,2 и является минимальным среди всех 10 отраслевых групп для компаний из этого индекса. Среднее соотношение этого показателя для всего индекса Standard & Poor's 500 (S&P 500) в сентябре составляет 16,2. Об этом сообщило агентство Bloomberg. Р/Е — один из основных показателей, использующийся аналитиками рынка для сравнительной оценки инвестиционного потенциала ценных бумаг. Чем больше этот показатель, тем выше цена акций по отношению к предполагаемой прибыли компании, что означает, что они «переоценены» относительно бумаг, с которыми сравниваются. Если две компании схожи по характеру, масштабам и видам деятельности и действуют на сходных рынках, то при прочих равных с высокой вероятностью можно ожидать, что их Р/Е со временем сблизятся. Вместе с тем «переоценка» или «недооценка» компании свидетельствует об уровне их привлекательности для инвес

In [62]:
get_response(TEXT, ft_index, modelFT, index_map)

['Отток иностранных денег из российских акций продолжается. За неделю, завершившуюся 23 января, иностранные фонды вывели с российского фондового рынка $61 млн. Это почти эквивалентно оттоку прошлой недели — $64 млн. Инвесторы выводят деньги с октября прошлого года. Небольшой приток за это время был зафиксирован лишь в первую рабочую неделю этого года — инвесторы вложили $56 млн. С начала 2013 года инвесторы вывели из российского фондового рынка $155 млн, следует из данных Emerging Portfolio Fund Research (EPFR). Основной отток обеспечили классические фонды — $47 млн, тогда как из спекулятивных фондов ETF было выведено $14 млн. «Активы под управлением российских страновых фондов сокращались в четырнадцати из последних шестнадцати недель — это резко контрастирует с непрерывным притоком средств в фонды Китая и развивающихся рынков в целом, продолжающимся уже на протяжении двадцати недель», — переживает Вячеслав Смольянинов , главный стратег «Уралсиб кэпитал». Россия по итогам недели показ

In [63]:
TEXT = "Добыча нефти"

In [64]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Цены на природный газ упали до 14-летнего минимума в США. На Нью-Йоркской торговой бирже стоимость газовых фьючерсов с поставками на январь в пересчете за тысячу кубометров снизилась до рекордных $68. По итогам торгов 14 декабря стоимость газа снизилась сразу на 4,8%, а в годовом исчислении цена упала на 34%, пишет издание Wall Street Journal. С учетом инфляции газ подешевел до 23-летнего минимума: цены на фьючерсы торговались на уровне $64,5 за тысячу кубометров лишь в феврале 1992 года. США являются крупнейшим производителем газа в мире (более 730 млрд кубометров в год). Добытый газ распространяется внутри страны: США являются и крупнейшим потребителем природного газа в мире (около 760 млрд кубометров в год). «Давление на газовые котировки в Америке оказывают сразу два фактора: очень теплая зима (температура на исторических максимумах), из-за чего прогнозируется снижение потребления в зимний период, и тот факт, что нефть, газойль и дизель — заменяющие газ виды топлива — сейчас на м

In [65]:
get_response(TEXT, ft_index, modelFT, index_map)

['«Газпром» не будет привязывать цену на газ для Китая к цене американских спотовых поставок. Об этом монополия сообщает во вторник по итогам встречи главы «Газпрома» Алексея Миллера и президента китайской CNPC Чжо Цзипина. Как рассказал «Газете.Ru» источник, близкий к «Газпрому», привязки цены к американскому Henry Hub добивались китайцы. В компании от комментариев отказались. Henry Hub — распределительный хаб в Луизиане, где сходятся восемь крупнейших американских газопроводов. Именно цена на Henry Hub служит ориентиром для определения цен по спотовым (разовым) газовым контрактам в США. За счет добычи газа из сланца США смогли полностью обеспечить собственные потребности и теперь планируют поставлять газ (в виде СПГ — сжиженного природного газа) на экспорт. В связи с этим цена газа в Америке значительно ниже, чем в Европе и тем более в Азии. «Цена спотовых контрактов в США на Henry Hub — $140-150 за 1 тыс. кубометров, — рассказывает директор по развитию бизнеса в странах СНГ агентств

#### ВЫВОД: в целом алгоритм работает и показывает удовлетворительный результат. Он хорошо справляется с запросами, носящими общий характер (Новости экономики). Но дает сбой когда запрос ставится более точный (Сырьевые рынки, Нефтяные рынки). В этом случае ключевым словом для алгоритма является слово "рынки". Это может быть сваязано с тем, что обучение проводилось не на полном тексте. Либо в корпусе недостаточно документов на эту тему.